## EJERCICIO 2

**Enunciado**: 
Usa la clase *PairWiseAligner* de Biopython para alinear secuencias
de ADN y obtener la puntuación y el alineamiento óptimo en estos dos escenarios.

**a)** Generar las secuencias aleatoriamente (de forma automática no manual).
Las dos secuencias pueden ser de diferente longitud.


In [151]:
from typing import Optional, Tuple, Iterable, List
import random
from dataclasses import dataclass
from pathlib import Path
from Bio import SeqIO
from Bio.Align import PairwiseAligner
import argparse

In [152]:
DNA_BASE = "AGCT"

In [153]:
@dataclass(frozen=True)
class GapScheme:
    linear_gap: Optional[float] = None
    open_gap: float = -10.0             
    extend_gap: float = -0.5             


@dataclass(frozen=True)
class AlignConfig:
    description: str
    mode: str               
    match_score: float = 2.0
    mismatch_score: float = -1.0
    gaps: GapScheme = GapScheme()

In [154]:
def generar_adn_aleatorio(longitud: int, seed: Optional[int] = None) -> str:
    if seed is not None:
        random.seed(seed)
    return "".join(random.choice(DNA_BASE) for _ in range(longitud))


def calcular_identidad_y_huecos(alA: str, alB: str) -> Tuple[float, int, int]:
    coincidencias = sum(1 for a, b in zip(alA, alB) if a == b and a != "-" and b != "-")
    posiciones_alineadas = sum(1 for a, b in zip(alA, alB) if a != "-" and b != "-")
    identidad = (coincidencias / posiciones_alineadas) * 100 if posiciones_alineadas else 0.0
    huecos = alA.count("-") + alB.count("-")
    return round(identidad, 2), coincidencias, huecos


def reconstruir_con_huecos(alignment, seq_a: str, seq_b: str) -> Tuple[str, str]:
    A, B = [], []
    coords = alignment.coordinates
    for j in range(coords.shape[1] - 1):
        a0, a1 = coords[0, j], coords[0, j + 1]
        b0, b1 = coords[1, j], coords[1, j + 1]

        while a0 < a1 and b0 < b1:
            A.append(seq_a[a0])
            B.append(seq_b[b0])
            a0 += 1
            b0 += 1
        while a0 < a1:
            A.append(seq_a[a0])
            B.append("-")
            a0 += 1
        while b0 < b1:
            A.append("-")
            B.append(seq_b[b0])
            b0 += 1
    return "".join(A), "".join(B)


def formatear_bloques(
    alA: str,
    alB: str,
    labelA: str = "Secuencia A",
    labelB: str = "Secuencia B",
    ancho: int = 60,
    off_a: int = 0,
    off_b: int = 0,
) -> str:
    i = 0
    out_lines = []
    while i < len(alA):
        sA = alA[i : i + ancho]
        sB = alB[i : i + ancho]
        mid = "".join(
            "|" if (a == b and a != "-" and b != "-") else ("." if a != "-" and b != "-" else " ")
            for a, b in zip(sA, sB)
        )
        startA = off_a + sum(1 for c in alA[:i] if c != "-") + 1
        endA = startA + sum(1 for c in sA if c != "-") - 1
        startB = off_b + sum(1 for c in alB[:i] if c != "-") + 1
        endB = startB + sum(1 for c in sB if c != "-") - 1

        out_lines.append(f"{labelA:<12} {startA:>6} {sA} {endA if endA>=startA else startA-1}")
        out_lines.append(f"{'':<12} {'':>6} {mid}")
        out_lines.append(f"{labelB:<12} {startB:>6} {sB} {endB if endB>=startB else startB-1}\n")
        i += ancho
    return "\n".join(out_lines).rstrip()

In [155]:
def configuration() -> list[AlignConfig]:
    return [
        AlignConfig(
            description="Global (lineal) m=2  mm=-1  gap=-1",
            mode="global",
            match_score=2.0,
            mismatch_score=-1.0,
            gaps=GapScheme(linear_gap=-1),
        ),
        AlignConfig(
            description="Global (lineal) m=2  mm=-1  gap=-5",
            mode="global",
            match_score=3.0,
            mismatch_score=-2.0,
            gaps=GapScheme(linear_gap=-5),
        ),
        AlignConfig(
            description="Local (lineal)  m=2  mm=-1  gap=-1",
            mode="local",
            match_score=4.0,
            mismatch_score=-2.0,
            gaps=GapScheme(linear_gap=-3),
        ),
        AlignConfig(
            description="Local (afín)    m=2  mm=-1  open=-8  ext=-1",
            mode="local",
            match_score=2.0,
            mismatch_score=-1.0,
            gaps=GapScheme(linear_gap=None, open_gap=-8, extend_gap=-1),
        ),
    ]

### Motor de alineamiento:

In [156]:
def ejecutar_alineamiento(
    seqA: str,
    seqB: str,
    config: AlignConfig,
    labelA: str = "Secuencia A",
    labelB: str = "Secuencia B",
    ancho: int = 60,
) -> str:
    aligner = PairwiseAligner()
    aligner.mode = config.mode
    aligner.match_score = config.match_score
    aligner.mismatch_score = config.mismatch_score

    if config.gaps.linear_gap is not None:
        aligner.open_gap_score = config.gaps.linear_gap
        aligner.extend_gap_score = config.gaps.linear_gap
    else:
        aligner.open_gap_score = config.gaps.open_gap
        aligner.extend_gap_score = config.gaps.extend_gap

    best = aligner.align(seqA, seqB)[0]

    alA, alB = reconstruir_con_huecos(best, seqA, seqB)
    identidad, matches, gaps = calcular_identidad_y_huecos(alA, alB)

    a_ini = best.aligned[0][0][0] if best.aligned[0].size else 0
    b_ini = best.aligned[1][0][0] if best.aligned[1].size else 0

    lines = []
    lines.append("\n" + "═" * 70)
    lines.append(f"Configuración: {config.description}")
    lines.append(f"Modo de alineamiento: {config.mode.upper()}")
    lines.append("─" * 70)
    lines.append(formatear_bloques(alA, alB, labelA, labelB, ancho, a_ini, b_ini))
    lines.append("─" * 70)
    lines.append(f"Puntuación total         : {best.score:.3f}")
    lines.append(f"Porcentaje de identidad  : {identidad}%")
    lines.append(f"Número de coincidencias  : {matches}")
    lines.append(f"Número total de huecos   : {gaps}")
    lines.append("═" * 70)
    return "\n".join(lines)


def comparar_alineamientos(
    seqA: str,
    seqB: str,
    titulo: str,
    labelA: str,
    labelB: str,
    configs: Iterable[AlignConfig],
) -> str:
    header = "\n" + "=" * 70 + f"\n{titulo.center(70)}\n" + "=" * 70
    bloques = [header]
    for cfg in configs:
        bloques.append(ejecutar_alineamiento(seqA, seqB, cfg, labelA=labelA, labelB=labelB))
    return "\n".join(bloques)

In [157]:
def main() -> None:
    import argparse

    def fmt(valor):
        if valor is None:
            return "N/A"
        return f"{valor:+g}"

    parser = argparse.ArgumentParser(description="Alineamiento de secuencias de ADN (aleatorias)")
    parser.add_argument("--lenA", type=int, default=40)
    parser.add_argument("--lenB", type=int, default=32)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--width", type=int, default=60)
    args = parser.parse_args([])

    seqA = generar_adn_aleatorio(args.lenA, seed=args.seed)
    seqB = generar_adn_aleatorio(args.lenB, seed=args.seed + 1)

    print("\n=== Secuencias generadas ===")
    print(f"A ({len(seqA)} nt): {seqA}")
    print(f"B ({len(seqB)} nt): {seqB}")

    configs = configuration()

    print("\n=== Resultados ===")
    for cfg in configs:
        match = cfg.match_score
        mismatch = cfg.mismatch_score
        gap_linear = cfg.gaps.linear_gap
        gap_open = cfg.gaps.open_gap
        gap_extend = cfg.gaps.extend_gap

        print("\nConfiguración:")
        print(f"- Coincidencia: {fmt(match)}")
        print(f"- No coincidencia: {fmt(mismatch)}")

        if gap_linear is not None:
            print(f"- Huecos: {fmt(gap_linear)}")
        else:
            print(f"- Huecos (abrir): {fmt(gap_open)}")
            print(f"- Huecos (extender): {fmt(gap_extend)}")

        print()  

        resultado = ejecutar_alineamiento(seqA, seqB, cfg, ancho=args.width)

        lineas = resultado.splitlines()
        lineas_filtradas = [
            linea for linea in lineas
            if not linea.strip().startswith("Configuración:")
            and not linea.strip().startswith("Modo de alineamiento:")
        ]
        resultado_limpio = "\n".join(lineas_filtradas)

        print(resultado_limpio)
        print("-" * 60)


if __name__ == "__main__":
    main()


=== Secuencias generadas ===
A (40 nt): AACGGGAATAAAGGAGTGTCAGTCCGGCAATACCCATATA
B (32 nt): ACGTCATTATCTTGAAATGAGTGTCGTAAAGA

=== Resultados ===

Configuración:
- Coincidencia: +2
- No coincidencia: -1
- Huecos: -1


══════════════════════════════════════════════════════════════════════
──────────────────────────────────────────────────────────────────────
Secuencia A       1 AACGGGA--AT----AAAGGAGTGTCAGTCCGGCAATACCCATATA 40
                    | ||..|  ||    |||.||||||| ||     || |.  |    
Secuencia B       1 A-CGTCATTATCTTGAAATGAGTGTC-GT-----AA-AG--A---- 32
──────────────────────────────────────────────────────────────────────
Puntuación total         : 20.000
Porcentaje de identidad  : 84.62%
Número de coincidencias  : 22
Número total de huecos   : 20
══════════════════════════════════════════════════════════════════════
------------------------------------------------------------

Configuración:
- Coincidencia: +3
- No coincidencia: -2
- Huecos: -5


══════════════════════════════

# Ejercicio 2 (a) - Resultados del alineamiento 

## Secuencias generadas

    A (40 nt): AACGGGAATAAAGGAGTGTCAGTCCGGCAATACCCATATA
    B (32 nt): ACGTCATTATCTTGAAATGAGTGTCGTAAAGA

## Resultados por configuración

### Configuración 1

-   Coincidencia: +2
-   No coincidencia: -1
-   Huecos: -1

```
```
    Secuencia A       1 AACGGGA--AT----AAAGGAGTGTCAGTCCGGCAATACCCATATA 40
                        | ||..|  ||    |||.||||||| ||     || |.  |    
    Secuencia B       1 A-CGTCATTATCTTGAAATGAGTGTC-GT-----AA-AG--A---- 32

**Resumen** - Puntuación total: 20.000 - Identidad: 84.62% -
Coincidencias: 22 - Huecos: 20

------------------------------------------------------------------------

### Configuración 2

-   Coincidencia: +3
-   No coincidencia: -2
-   Huecos: -5

```
```
    Secuencia A       1 AACGGGAATAAAG-GAGTGTCAGTCCGGCAATACCCATATA 40
                        | ||..|.||... ||.. |.|||  |.|. ||   | |.|
    Secuencia B       1 A-CGTCATTATCTTGAAA-TGAGT--GTCG-TA---A-AGA 32

**Resumen** - Puntuación total: -17.000 - Identidad: 61.29% -
Coincidencias: 19 - Huecos: 10

------------------------------------------------------------------------

### Configuración 3

-   Coincidencia: +4
-   No coincidencia: -2
-   Huecos: -3

```
```
    Secuencia A       2 ACGGGA--AT----AAAGGAGTGTCAGT 23
                        |||..|  ||    |||.||||||| ||
    Secuencia B       1 ACGTCATTATCTTGAAATGAGTGTC-GT 27

**Resumen** - Puntuación total: 45.000 - Identidad: 85.71% -
Coincidencias: 18 - Huecos: 7

------------------------------------------------------------------------

### Configuración 4

-   Coincidencia: +2
-   No coincidencia: -1
-   Huecos (abrir): -8
-   Huecos (extender): -1

```
```
    Secuencia A      10 AAAGGAGTGTC 20
                        |||.|||||||
    Secuencia B      15 AAATGAGTGTC 25

**Resumen** - Puntuación total: 19.000 - Identidad: 90.91% -
Coincidencias: 10 - Huecos: 0


---
#### BREVE RESUMEN: 
En este apartado probamos diferentes configuraciones del alineamiento cambiando la penalización por huecos y las puntuaciones de coincidencia.

Cuando poner un hueco es barato (por ejemplo, gap = -1), el alineador introduce muchos huecos porque así puede hacer coincidir más letras. Como resultado, la identidad obtenida es alta, aunque el alineamiento tenga muchos cortes.

Cuando los huecos son muy costosos (por ejemplo, gap = -5), el alineador intenta evitarlos. Prefiere dejar algunas letras sin coincidir antes que abrir huecos. Esto produce menos huecos, pero también baja la identidad entre las secuencias.

En el modo local, el alineador solo busca la región donde las secuencias son más parecidas. Ignora los extremos y se queda con el fragmento más alineable. Por eso se obtienen porcentajes de identidad muy altos, especialmente cuando abrir huecos es muy caro (open = -8). En estos casos, el alineador tiende a formar un bloque continuo sin insertar gaps.

---


**b)** Obtener las secuencias a partir de ficheros obtenidos de bases de
datos biológicas. Explicar cómo se obtuvieron.
Prueba a cambiar los parámetros del alineador como el esquema de
puntuación, las penalizaciones de alineamiento y la modalidad de
alineamiento (global o local). Compara los resultados obtenidos
con diferentes configuraciones y explica las diferencias.

In [158]:
DEFAULT_FASTA_DIR = Path("./fasta")
DEFAULT_MOUSE_FASTA = DEFAULT_FASTA_DIR / "mouse.fasta"
DEFAULT_LIZARD_FASTA = DEFAULT_FASTA_DIR / "lizard.fasta"

In [159]:
@dataclass(frozen=True)
class GapScheme:
    linear_gap: Optional[float] = None
    open_gap: float = -10.0
    extend_gap: float = -0.5


@dataclass(frozen=True)
class AlignConfig:
    description: str
    mode: str
    match_score: float = 2.0
    mismatch_score: float = -1.0
    gaps: GapScheme = GapScheme()

### Métodos implementados para poder realizar el alineamiento: 

In [160]:
def leer_secuencia_fasta(path: Path) -> str:
    if not path.exists():
        raise FileNotFoundError(f"No se encontró el archivo FASTA: {path}")
    registro = next(SeqIO.parse(str(path), "fasta"))
    return str(registro.seq).upper().replace("U", "T")

def calcular_identidad_y_huecos(alA: str, alB: str) -> Tuple[float, int, int]:
    coincidencias = sum(
        1 for a, b in zip(alA, alB) if a == b and a != "-" and b != "-"
    )
    posiciones_alineadas = sum(
        1 for a, b in zip(alA, alB) if a != "-" and b != "-"
    )
    identidad = (coincidencias / posiciones_alineadas) * 100 if posiciones_alineadas else 0.0
    huecos = alA.count("-") + alB.count("-")
    return round(identidad, 2), coincidencias, huecos


def reconstruir_con_huecos(alignment, seq_a: str, seq_b: str) -> Tuple[str, str]:
    A, B = [], []
    coords = alignment.coordinates
    for j in range(coords.shape[1] - 1):
        a0, a1 = coords[0, j], coords[0, j + 1]
        b0, b1 = coords[1, j], coords[1, j + 1]
        while a0 < a1 and b0 < b1:
            A.append(seq_a[a0])
            B.append(seq_b[b0])
            a0 += 1
            b0 += 1
        while a0 < a1:
            A.append(seq_a[a0])
            B.append("-")
            a0 += 1
        while b0 < b1:
            A.append("-")
            B.append(seq_b[b0])
            b0 += 1
    return "".join(A), "".join(B)


def formatear_bloques(
    alA: str,
    alB: str,
    labelA: str = "Secuencia A",
    labelB: str = "Secuencia B",
    ancho: int = 60,
    off_a: int = 0,
    off_b: int = 0,
) -> str:
    i = 0
    out_lines = []
    while i < len(alA):
        sA = alA[i : i + ancho]
        sB = alB[i : i + ancho]
        mid = "".join(
            "|" if (a == b and a != "-" and b != "-") else ("." if a != "-" and b != "-" else " ")
            for a, b in zip(sA, sB)
        )
        startA = off_a + sum(1 for c in alA[:i] if c != "-") + 1
        endA = startA + sum(1 for c in sA if c != "-") - 1
        startB = off_b + sum(1 for c in alB[:i] if c != "-") + 1
        endB = startB + sum(1 for c in sB if c != "-") - 1

        out_lines.append(f"{labelA:<12} {startA:>6} {sA} {endA}")
        out_lines.append(f"{'':<12} {'':>6} {mid}")
        out_lines.append(f"{labelB:<12} {startB:>6} {sB} {endB}\n")
        i += ancho
    return "\n".join(out_lines).rstrip()

### Motor de alineamiento:

In [161]:
def ejecutar_alineamiento(
    seqA: str,
    seqB: str,
    config: AlignConfig,
    labelA: str = "Secuencia A",
    labelB: str = "Secuencia B",
    ancho: int = 60,
) -> str:
    aligner = PairwiseAligner()
    aligner.mode = config.mode
    aligner.match_score = config.match_score
    aligner.mismatch_score = config.mismatch_score

    if config.gaps.linear_gap is not None:
        aligner.open_gap_score = config.gaps.linear_gap
        aligner.extend_gap_score = config.gaps.linear_gap
    else:
        aligner.open_gap_score = config.gaps.open_gap
        aligner.extend_gap_score = config.gaps.extend_gap

    best = aligner.align(seqA, seqB)[0]

    alA, alB = reconstruir_con_huecos(best, seqA, seqB)
    identidad, matches, gaps = calcular_identidad_y_huecos(alA, alB)

    a_ini = best.aligned[0][0][0] if best.aligned[0].size else 0
    b_ini = best.aligned[1][0][0] if best.aligned[1].size else 0

    lines = []
    lines.append("\n" + "═" * 70)
    lines.append(f"Configuración: {config.description}")
    lines.append(f"Modo de alineamiento: {config.mode.upper()}")
    lines.append("─" * 70)
    lines.append(formatear_bloques(alA, alB, labelA, labelB, ancho, a_ini, b_ini))
    lines.append("─" * 70)
    lines.append(f"Puntuación total         : {best.score:.3f}")
    lines.append(f"Porcentaje de identidad  : {identidad}%")
    lines.append(f"Número de coincidencias  : {matches}")
    lines.append(f"Número total de huecos   : {gaps}")
    lines.append("═" * 70)
    return "\n".join(lines)


def comparar_alineamientos(
    seqA: str,
    seqB: str,
    titulo: str,
    labelA: str,
    labelB: str,
    configs: Iterable[AlignConfig],
) -> str:
    header = "\n" + "=" * 70 + f"\n{titulo.center(70)}\n" + "=" * 70
    bloques = [header]
    for cfg in configs:
        bloques.append(ejecutar_alineamiento(seqA, seqB, cfg, labelA=labelA, labelB=labelB))
    return "\n".join(bloques)

In [162]:
def configuration() -> list[AlignConfig]:
    return [
        AlignConfig(
            description="Global (afín)   m=2  mm=-1  open=-10  ext=-0.5",
            mode="global",
            gaps=GapScheme(linear_gap=None, open_gap=-10, extend_gap=-0.5),
        ),
        AlignConfig(
            description="Global (lineal) m=2  mm=-1  gap=-2",
            mode="global",
            gaps=GapScheme(linear_gap=-2),
        ),
        AlignConfig(
            description="Local (afín)    m=2  mm=-1  open=-10  ext=-0.5",
            mode="local",
            gaps=GapScheme(linear_gap=None, open_gap=-10, extend_gap=-0.5),
        ),
        AlignConfig(
            description="Local (lineal)  m=2  mm=-1  gap=-2",
            mode="local",
            gaps=GapScheme(linear_gap=-2),
        ),
    ]

In [163]:
def main() -> None:
    parser = argparse.ArgumentParser(
        description="Ejercicio 2: Alineamiento de secuencias con PairwiseAligner (ADN)"
    )
    parser.add_argument(
        "--fasta",
        nargs=2,
        metavar=("SEQ1.fasta", "SEQ2.fasta"),
        help="Alinear dos secuencias obtenidas desde archivos FASTA",
    )
    parser.add_argument("--lenA", type=int, default=40, help="Longitud de la primera secuencia aleatoria")
    parser.add_argument("--lenB", type=int, default=32, help="Longitud de la segunda secuencia aleatoria")
    args = parser.parse_args([])

    if args.fasta:
        fasta1, fasta2 = map(Path, args.fasta)
    else:
        fasta1, fasta2 = DEFAULT_MOUSE_FASTA, DEFAULT_LIZARD_FASTA

    try:
        seqA = leer_secuencia_fasta(fasta1)
        seqB = leer_secuencia_fasta(fasta2)
        labelA = fasta1.stem
        labelB = fasta2.stem
        titulo = "(b) Secuencias obtenidas de bases de datos"
        print("\nArchivos FASTA cargados correctamente:")
        print(f" - {fasta1.name} (longitud: {len(seqA)})")
        print(f" - {fasta2.name} (longitud: {len(seqB)})")
    except FileNotFoundError as e:
        print(f"\n{e}\nNo se encontraron los FASTA; se usarán secuencias aleatorias.")
        seqA = generar_adn_aleatorio(args.lenA)
        seqB = generar_adn_aleatorio(args.lenB)
        labelA, labelB = "Secuencia A", "Secuencia B"
        titulo = "(a) Secuencias aleatorias"
        print("\nSecuencias aleatorias generadas:")
        print(f" - {labelA} (longitud {len(seqA)}): {seqA}")
        print(f" - {labelB} (longitud {len(seqB)}): {seqB}")

    configs = configuration()
    reporte = comparar_alineamientos(seqA, seqB, titulo=titulo, labelA=labelA, labelB=labelB, configs=configs)
    print(reporte)


if __name__ == "__main__":
    main()


Archivos FASTA cargados correctamente:
 - mouse.fasta (longitud: 3640)
 - lizard.fasta (longitud: 93773)

              (b) Secuencias obtenidas de bases de datos              

══════════════════════════════════════════════════════════════════════
Configuración: Global (afín)   m=2  mm=-1  open=-10  ext=-0.5
Modo de alineamiento: GLOBAL
──────────────────────────────────────────────────────────────────────
mouse             1 ------------------------------------------------------------ 0
                                                                                
lizard           90 CGCGGTAGCCTGCTTGGTGGAGACTGGGTGCGCCTGCGTACTTCATAGTTCGCGTAGCGG 149

mouse             1 -----------------------------TATAAAACCC---------GGCGGCGCAACG 22
                                                 ||.|||.|||         |||||...||.|
lizard          150 CTCGAGCCTGGAGATGAAGGTAAATAACTTACAAATCCCCGGGCGGGGGGCGGGAAAATG 209

mouse            23 C----------------------------------------------------------- 23
   